# Assignment 2: Jetty Scheduling

Name: **Moritz Grävinghoff, Frederic Weiss, Christian Husmann**\
Data of submission: **01.04.2024**

## Question 2

### c) Implementation

#### Import all required packages

In [25]:
import pandas as pd
from docplex.mp.model import Model
from itertools import product
from plotly import figure_factory as ff

#### 1. Load data

In [26]:
def load_data(problem_nr):
    
    instance = pd.ExcelFile(f'../instances/inst{problem_nr}.xlsx')

    ship_names = instance.parse(f'Problem_{problem_nr}', header=1, usecols='G').iloc[:,0].to_list()
    ships = list(range(len(ship_names)))
    categories = instance.parse(f'Problem_{problem_nr}', header=1, usecols='H').iloc[:,0].to_list()
    arrivals = instance.parse(f'Problem_{problem_nr}', header=1, usecols='K').iloc[:,0].to_list()

    # As we start indexing by 0, decrease categories and arrivals by 1
    categories = [x-1 for x in categories]
    arrivals = [x-1 for x in arrivals]

    unloading_times = instance.parse(f'Problem_{problem_nr}', header = 1, usecols = 'B:E').dropna()
    unloading_times = unloading_times.values.tolist()
    
    # Define stations
    stations = list(range(4))

    return ships, ship_names, categories, arrivals, stations, unloading_times

#### 2. Create model instance

In [27]:
def create_model(ships, categories, arrivals, stations, unloading_times):

    # Create model
    model = Model(name='jetty_scheduling')

    # Define decision variables
    X = model.integer_var_dict(product(ships,stations), name='starting') # first period that ship s is located on station p
    W = model.integer_var_dict(product(ships,stations), name='waiting') # periods that ship s waits at station p before moving to next station
    A = model.binary_var_dict(product(ships,stations), name='assignment') # 1 if ship s is assigned to station p for unloading, 0 otherwise
    E = model.binary_var_dict(product(ships,ships), name='order') # 1 if ship s enters the jetty before ship s_bar, 0 otherwise
    F = model.integer_var_dict(ships, name='finish') # finishing time of ship s
    
    # Define constraints
    bigM = 9999

    # (1) each ship has to be assigned to exactly one station for unloading
    model.add_constraints(model.sum(A[s,p] for p in stations) == 1 for s in ships)

    # (2) no movement to first station before arrival
    model.add_constraints(X[s,0] >= arrivals[s] for s in ships)

    # (3) stay at assigned station for unloading, else for at least one period
    model.add_constraints(X[s,p+1] == X[s,p] + A[s,p] * unloading_times[categories[s]][p] + (1-A[s,p]) + W[s,p] for s in ships for p in stations if p<3)

    # (4) move to next station only if previous ship has already left
    model.add_constraints(X[s_bar,p] >= X[s,p] + A[s,p] * unloading_times[categories[s]][p] + (1-A[s,p]) + W[s,p] - E[s_bar,s] * bigM for s in ships for s_bar in ships for p in stations if s!=s_bar)

    # (5) ship is either before or after every other ship
    model.add_constraints(1 == E[s,s_bar] + E[s_bar,s] for s in ships for s_bar in ships if s!=s_bar)

    # (6) finishing time of each ship
    model.add_constraints(F[s] == X[s,3] + A[s,3] * unloading_times[categories[s]][3] + (1-A[s,3]) for s in ships)

    # Define objective function: minimize total sum of periods all ships are in waiting position or at the jetty
    model.minimize(model.sum(F[s] - arrivals[s] for s in ships))

    return model, X, W, A, E, F

#### 3. Create Gantt chart

In [28]:
def create_gantt_chart(problem_nr, model, ships, ship_names, stations, X, F):

    # Dataframe to store information for gantt chart
    df_gantt = pd.DataFrame(columns=['Task','Start','Finish','Resource'])

    # For all ships, add periods at each station to dataframe
    for s in ships:
        for p in stations[:-1]:
            # first three stations
            df_gantt = pd.concat([df_gantt, pd.DataFrame({'Task': f'Station {p+1}', 'Start': [X[s,p].solution_value], 'Finish': [X[s,p+1].solution_value], 'Resource': ship_names[s]})], ignore_index=True)
        # last station
        df_gantt = pd.concat([df_gantt, pd.DataFrame({'Task': f'Station {4}', 'Start': [X[s,3].solution_value], 'Finish': [F[s].solution_value], 'Resource': ship_names[s]})], ignore_index=True)

    # Get finishing time
    finish = int(max([F[s].solution_value for s in ships]))

    # Create gantt chart
    fig = ff.create_gantt(df_gantt, index_col='Resource', show_colorbar=True, bar_width=0.4, group_tasks=True, title=f'Gantt Chart for Instance {problem_nr}; Finished after {finish} periods; Objective value: {int(model.objective_value)}')
    fig.update_layout(xaxis_type='linear', autosize=False, width=1000)

    # Return gantt chart
    return fig

#### 4. Export results to Excel file

In [29]:
def export_results():

    return 

### d) Run model for all problem instances

In [30]:
for problem_nr in range(1,4):

    # Load data from excel file
    ships, ship_names, categories, arrivals, stations, unloading_times = load_data(problem_nr)

    # Create model instance
    model, X, _, _, _, F = create_model(ships, categories, arrivals, stations, unloading_times)

    # Solve the model
    model.solve()

    # Export results to excel file
    export_results()

    # Create gantt chart
    gantt = create_gantt_chart(problem_nr, model, ships, ship_names, stations, X, F)

    display(gantt)

C:\Users\mogra\AppData\Local\Temp\ipykernel_15772\2570651001.py:10: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



C:\Users\mogra\AppData\Local\Temp\ipykernel_15772\2570651001.py:10: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



C:\Users\mogra\AppData\Local\Temp\ipykernel_15772\2570651001.py:10: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

